### LangChain Intro — Amdocs OpenAI Gateway

This notebook uses the **Amdocs OpenAI Gateway** to connect to an OpenAI-style LLM (e.g. gpt-4.1).

**Endpoint:** `https://ai-framework1:8085/api/v1/call_llm`  
**Required in `.env`** (in project root, e.g. `RAG_UDEMY/.env`):  
- `AMDOCS_API_KEY` — your API key  
- `AMDOCS_EFFECTIVE_CALLER` — e.g. `abhisha3@amdocs.com`  

If you see "Invalid token", regenerate your token in the Amdocs LLM API portal and update `.env`.

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env from notebook dir and from project root (parent of langchain/)
_cwd = Path(os.getcwd())
load_dotenv(_cwd / ".env")
load_dotenv(_cwd.parent / ".env")

AMDOCS_API_KEY = os.getenv("AMDOCS_API_KEY") or os.getenv("OPEN_AI_KEY")
AMDOCS_EFFECTIVE_CALLER = os.getenv("AMDOCS_EFFECTIVE_CALLER", "abhisha3@amdocs.com")

print("Amdocs Gateway:", "Ready" if AMDOCS_API_KEY else "Not configured (set AMDOCS_API_KEY in .env)")
print("Effective caller:", AMDOCS_EFFECTIVE_CALLER)

Amdocs Gateway: Ready


#### Basic agent (no tools)

Create the LLM via `AmdocsChatOpenAI` and a simple LangChain agent.

In [2]:
import sys
import os

# Ensure langchain folder (where amdocs_chat.py lives) is on path
_cwd = os.getcwd()
_langchain_dir = _cwd if "langchain" in _cwd else os.path.join(_cwd, "langchain")
for _d in (_langchain_dir, _cwd):
    if _d and os.path.isdir(_d) and _d not in sys.path:
        sys.path.insert(0, _d)

from langchain.agents import create_agent
from amdocs_chat import AmdocsChatOpenAI

if AMDOCS_API_KEY:
    amdocs_llm = AmdocsChatOpenAI(
        api_key=AMDOCS_API_KEY,
        effective_caller=AMDOCS_EFFECTIVE_CALLER,
        llm_model="gpt-4.1",
        max_tokens=8000,
        verify_ssl=False,
    )
    agent_amdocs = create_agent(
        model=amdocs_llm,
        tools=[],
        system_prompt="You are helpful.",
    )
    print("Amdocs agent created. Invoke with: agent_amdocs.invoke({'messages': [...]})")
else:
    agent_amdocs = None
    amdocs_llm = None
    print("Set AMDOCS_API_KEY or OPEN_AI_KEY in .env, then re-run the env cell above.")

Amdocs agent created. Invoke with: agent_amdocs.invoke({'messages': [...]})


In [3]:
# Test the basic Amdocs agent (use "messages" as the input key)
if agent_amdocs:
    try:
        out = agent_amdocs.invoke({"messages": [{"role": "user", "content": "Say hello in one sentence."}]})
        last = out.get("messages", [])[-1]
        print(last.content if hasattr(last, "content") and isinstance(last.content, str) else out)
    except Exception as e:
        print("Error:", type(e).__name__, str(e))
        raise

Error: RuntimeError Amdocs gateway error 404: 404 Client Error: Not Found for url: https://ai-framework1:8085/api/v1/call_llm. Response: {"detail":"Invalid token. Please regenerate your token to continue making requests to the LLM APIs."}


RuntimeError: Amdocs gateway error 404: 404 Client Error: Not Found for url: https://ai-framework1:8085/api/v1/call_llm. Response: {"detail":"Invalid token. Please regenerate your token to continue making requests to the LLM APIs."}

#### Agent with weather tool

Define a simple `get_weather` tool and create an agent that can use it.

In [ ]:
def get_weather(city: str) -> str:
    """Get the weather in a city."""
    return f"The weather in {city} is sunny."

In [ ]:
if amdocs_llm:
    agent_amdocs_weather = create_agent(
        model=amdocs_llm,
        tools=[get_weather],
        system_prompt="You are helpful. Use the get_weather tool when the user asks about weather in a city.",
    )
    print("Amdocs agent with weather tool created.")
else:
    agent_amdocs_weather = None

Amdocs agent with weather tool created.


In [ ]:
# Invoke with a weather question (input key must be "messages")
if agent_amdocs_weather:
    out = agent_amdocs_weather.invoke({
        "messages": [{"role": "user", "content": "What is the weather in Mumbai?"}]
    })
    print(out.get("messages", [])[-1].content if isinstance(out.get("messages", [])[-1].content, str) else out)

NotImplementedError: 